## PyTorch mnist 實例演練

程式執行時間約為

- 141.72 sec (with GPU: GeForce GTX 860M)

- 680.75 sec (with CPU: i5-4210H)

In [ ]:
# 首先載入需要用的到套件
# 矩陣模組運算工具
import numpy as np                              
# 繪圖工具
import matplotlib.pyplot as plt                 

import torch
# PyTorch 的神經網路套件 (具參數調整)
import torch.nn as nn                           
# PyTorch 的神經網路套件 (不具參數調整)
import torch.nn.functional as F                 
# PyTorch 用作最佳化的套件
import torch.optim as optim                     
# PyTorch 電腦視覺相關套件
import torchvision                              
# PyTorch 電腦視覺相關套件
from torchvision import datasets, transforms    

In [ ]:
# 建立CNN模型
class Net(nn.Module):
    # 我們這邊實作具兩層卷積層及兩層全連接層的CNN
    def __init__(self):
        super(Net, self).__init__()
        # 卷積層 1: nn.Conv2d(in_channels, out_channels, kernel_size, stride=1)
        self.conv1 = nn.Conv2d(1, 20, 5, 1)    
        # 卷積層 2: nn.Conv2d(in_channels, out_channels, kernel_size, stride=1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)   
        # 全連接層 1:　nn.Linear(in_features, out_features）
        self.fc1 = nn.Linear(4*4*50, 500)      
        # 全連接層 2 (最後接上 10 類別分類):　nn.Linear(in_features, out_features）
        self.fc2 = nn.Linear(500, 10)          
    
    # 模型中的層傳遞
    def forward(self, x):
        # 對 input x 做 conv1 的 convolution 後以 ReLU 啟動函數進行處理
        x = F.relu(self.conv1(x))              
        # 對接續的 x 做 2 * 2 的 max pooling
        x = F.max_pool2d(x, 2, 2)              
        # 對接續的 x 做 conv2 的 convolution 後以 ReLU 啟動函數進行處理
        x = F.relu(self.conv2(x))              
        # 對接續的 x 做 2 * 2 的 max pooling
        x = F.max_pool2d(x, 2, 2)              
        # 改變張量維度 (flatten) 以接續全連接層
        x = x.view(-1, 4*4*50)                 
        # 對接續的 x 做 fc1 全連接，並以ReLU啟動函數進行處理
        x = F.relu(self.fc1(x))                
        # 對接續的 x 做 fc2 全連接，並以ReLU啟動函數進行處理
        x = self.fc2(x)                        
        # return softmax 函數作類別分類
        return F.log_softmax(x, dim=1)         

In [ ]:
# 超參數區 (hyperparameters)
# 本區參數可手動調整以最佳化模型

# 進行一次批次訓練所使用的資料量
batch_size = 64           
# 進行一次批次驗證所使用的資料量
test_batch_size = 1000    
# 訓練次數 = 10
epochs = 10               
# 模型的學習率
lr = 0.01                 
# 模型最佳化時用以調整學習率的參數
momentum = 0.5            
# 每 10 次迭代就印出一次當前訓練狀態
log_interval = 10         

In [ ]:
# 設定 torch 進行運算的裝置，如果有 cuda 就使用 GPU 進行運算，若無則使用 CPU 進行運算
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 載入手寫字辨識訓練資料
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        './data',
        # 表示進行訓練 (訓練過程調整模型參數)
        train=True,                                              
        # 若之前無下載過 mnist，則會自動進行下載資料的動作
        download=True,                                           
        # 表示將影像資料轉為張量進行運算
        transform=transforms.Compose([transforms.ToTensor()])
    ),  
    batch_size=batch_size, shuffle=True
)

In [ ]:
# 載入手寫字辨識驗證資料
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        './data',
        # 表示進行驗證 (不調整模型參數)
        train=False,                                             
        # 表示將影像資料轉為張量進行運算
        transform=transforms.Compose([transforms.ToTensor()])
    ),  
    batch_size=test_batch_size, shuffle=False
)

In [ ]:
# 定義訓練過程
def train(model, device, train_loader, optimizer, epoch):
    # 開始訓練
    model.train()
    # 從 train_loader 中取出 batch 的 index, 資料, 以及資料的標籤
    for batch_idx, (data, target) in enumerate(train_loader):   
        # 指定不同變數之張量於運算的GPU或CPU裝置
        data, target = data.to(device), target.to(device)       
        # 模型初始化
        optimizer.zero_grad()                
        # 將資料傳入模型
        output = model(data)                 
        # 計算模型預測的答案與真實資料的誤差(negative log likelihood loss)
        loss = F.nll_loss(output, target)    
        # 反向傳播
        loss.backward()                      
        # 調整模型內部參數
        optimizer.step()                     
        # print 出該模型在 epoch 的訓練資料百分比以及 loss 數值
        if batch_idx % log_interval == 0:    
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
# 定義驗證過程
def test(model, device, test_loader):
    # 開始驗證
    model.eval()
    # 初始化 loss
    test_loss = 0    
    # 初始化預測正確數
    correct = 0      
    #驗證過程不進行模型內部參數調整
    with torch.no_grad():     
        # 從 test_loader 中取出資料以及資料的標籤
        for data, target in test_loader:                          
            # 指定不同變數之張量於運算的 GPU 或 CPU 裝置
            data, target = data.to(device), target.to(device)     
            # 將資料傳入模型
            output = model(data)                                  
            # 計算驗證過程的 loss 數值
            test_loss += F.nll_loss(output, target, reduction='sum').item()  
            # 以模型預測的分數的指標位置去取得預測的答案(類別)
            pred = output.argmax(dim=1, keepdim=True)                        
            # 計算模型預測的正確率
            correct += pred.eq(target.view_as(pred)).sum().item()            

    # 平均批次驗證的 loss 數值
    test_loss /= len(test_loader.dataset)                                    

    # print 出 loss 數值及正確率
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format( 
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
# 將模型指定到 GPU 或 CPU 裝置進行運算
model = Net().to(device)            
# model = nn.DataParallel(model)    # 利用nn.DataParallel來使模型以多GPU進行運算
# 使用stochastic gradient descent進行最佳化
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)     

In [ ]:
# 以 for 迴圈進行訓練及驗證
# range 的 epoch 數字要 +1
for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

# 儲存訓練好的模型
torch.save(model.state_dict(),"mnist_cnn.pt") 

In [ ]:
# 接著取出一些手寫字辨識的影像來查看預測情況

# 將 test_loader 的資料取出且以跟前面一樣的裝置做張量儲存
examples = enumerate(test_loader)    
batch_idx, (test_data, test_targets) = next(examples)
test_data = test_data.to(device)

In [ ]:
# 將資料傳入模型 (在此屬於驗證過程)
with torch.no_grad():
    output = model(test_data)

In [ ]:
# 以 matplotlib 做 3 * 3 的組合圖
fig = plt.figure()
for k in range(9):
    plt.subplot(3,3,k+1)
    # 優化組合圖排列
    plt.tight_layout()    
    # 以灰階影像顯示出每一張子圖
    plt.imshow(test_data[k][0].cpu().numpy(), cmap='gray')                              
    # 將預測的答案設為每個子圖的 title
    plt.title("Prediction: {}".format(output.data.max(1, keepdim=True)[1][k].item()))   
    # 不顯示X座標軸刻度
    plt.xticks([])        
    # 不顯示Y座標軸刻度
    plt.yticks([])        